In [206]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import skewnorm
import random
# Import custom classes and helper functions 

### Helper functions

In [207]:
def seq_along(list: list):
    return range(len(list))

def sort_dict(dictionary):
    sorted_d = dict(sorted(dictionary.items()))
    return sorted_d

def sample_skew(skew=4, mean=7, sd=15):
    skewed_dist = skewnorm(skew, mean, sd)
    skew_num = skewed_dist.rvs()
    skew_num_round = round(skew_num, 0)
    if skew_num_round < 1:
        return random.randint(1, 6)
    else: 
        return skew_num_round

### Grocery Classes
1. Customer
2. Cashier
3. Register
4. Store

In [208]:
class Customer:
    
    def __init__(self, name: str, store, n: int, vessel: str):
        self.name = name
        self.items = n
        self.store = store
        self.position = random.randint(0, store.n_registers)

        if vessel in ["cart", "basket"]:
            self.vessel = vessel
        else:
            raise ValueError("`vessel` must be one of 'cart' or 'basket'")

    def choose_register(self):

        # DISTANCE IS RELATIVE TO CUSTOMER
        distance_arr = np.array([np.abs(self.position - reg.index) for reg in self.store.registers]) 
        distance_arr = np.exp(-distance_arr)

        line_arr = np.array([reg.line_length for reg in self.store.registers])
        line_arr = np.exp(-line_arr)

        item_arr = np.array([reg.items for reg in self.store.registers])
        item_arr = np.exp(-item_arr)

        speed_arr = np.array([reg.cashier.speed for reg in self.store.registers])
        speed_arr = speed_arr / max(speed_arr)

        attr_matrix = np.column_stack((distance_arr, line_arr, item_arr, speed_arr))

        # Add ability vector
        # TODO: design ability vector (i.e., each distribution) 
        attr_matrix += [1, 6, 100, 0]
        
        # weighting
        # TODO: design weights vector 
        weighted_attrs = np.multiply(attr_matrix, [random.randint(1, 100), random.randint(1, 100), random.randint(1, 100), random.randint(1, 100)])

        # Sum up attribute values by 
        summed = np.sum(weighted_attrs, axis=1)
        chosen_register = np.argmax(summed)
        
        # Join chosen register
        register = self.store.registers[chosen_register]

        register.line_length += 1
        register.items += self.items
        register.vessel.append(self.vessel)
        register.customers.append(self.name)

In [193]:
class Cashier:
    def __init__(self, speed: int):
        self.speed = speed

In [194]:
class Register:
    
    bagger_multiplier = 1.5

    def __init__(self, index: int, cashier, bagger: bool):
        """A Register instance

        Args:
            index (int): unique identifier
            cashier (Cashier): instance of class Cashier
            bagger (bool): whether register includes a bagger
        """
        self.index = index
        # ! New register always has 0 items, 0 customers in line, and is active
        self.items = 0
        self.line_length = 0
        self.customers = []
        self.vessel = []
        self.active = True
    
        if cashier is None:
            self.cashier = None
        else:
            self.cashier = cashier
        # A bagger is optional (you can always add one later with add_bagger())
            self.bagger = bagger
        # If a bagger is added, checkout speed increases
            if bagger:
                self.speed = cashier.speed * self.bagger_multiplier
            else:
                self.speed = cashier.speed

    def activate(self):
        self.active = True

    def deactivate(self):
        self.active = False
    
    def add_bagger(self):
    # TODO: since baggers are just normal employees, add: check Store has enough employees free
        self.bagger = True
        self.speed = self.speed * self.bagger_multiplier

In [ ]:
class Store:
    
    def __init__(self, n_registers: int, employees: int):
        
        registers = []
        free_employees = employees

        for i in range(n_registers):
            if free_employees <= 0:
                current_register = Register(index=i, cashier=None, bagger=False)
                current_register.deactivate()
                registers.append(current_register)

            else:
                cashier = Cashier(np.random.normal(30, 5))
                current_register = Register(index=i, cashier = cashier, bagger = False)
                registers.append(current_register)
            
            free_employees -= 1

        self.registers = registers
        self.n_registers = n_registers
        self.employees = employees

### Check-out flow

1. Customer approaches the register area
2. Customer queues at the register with the shortest line length
3. **Later feature**: While the customer is at the top of the stack, they can switch to another register with shorter line or fewer items, since now they have had a bit of time to survey the other registers' lines and the contents of their customers' carts
4. Each register processes customers at a rate of n seconds per item, which is a fixed cashier trait
   a. The processing rate can be increased by a constant multiplier if a bagger is added to register
   b. When register is empty (0 items), cashier can "add" a customer from a neighboring queue (FIFO)  
5. The store can open another register if it has the capacity 

In [219]:
cub = Store(15, 30)

miriam = Customer(name="miriam", store=cub, n=30, vessel="cart")
lizzie = Customer("lizzie", cub, 70, "cart")
miriam.choose_register()
miriam.choose_register()
lizzie.choose_register()
